# Opening and cleaning out Bin data

Note: Use L1imag/formation environment.
## Import packages

In [ ]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
%matplotlib widget

from ephyviewer import mkQApp, MainViewer, TraceViewer

## Load data

WARNING: Absurd here, just temporary loading to see if and how I can access the data

In [ ]:
folder = Path('.').absolute()

list_raw = []
for file_path in folder.glob('**/*.bin'):
    recording = file_path.stem.replace('20211014_OPCA_031TR_BRstimON_Part1.nidq','1')
    globals()[f"DataRec_{recording}"] = np.fromfile(file_path, dtype="int16")
    globals()[f"DataRaw_{recording}"] = globals()[f"DataRec_{recording}"].reshape(-1,12)
    list_raw.append(f"DataRaw_{recording}")


### Distribute channels.

EMG = 0, BLA left = 1-2, BLA right = 3-4, PFC left = 5-6, PFC right = 9-10, CA1 = 7-8, TTL = 11

WARNING: this has to be adjusted for every mouse. In the future, add a cell at the beginning to enter these parameters

In [ ]:
list_rawchan = []
for ele, file in enumerate(list_raw):
    ele2 = ele + 1
    # Distribute
    EMG = globals()[f"DataRaw_{ele2}"][:,0]
    BLAl = globals()[f"DataRaw_{ele2}"][:,1] - globals()[f"DataRaw_{ele2}"][:,2]
    BLAr = globals()[f"DataRaw_{ele2}"][:,3] - globals()[f"DataRaw_{ele2}"][:,4]
    PFCl = globals()[f"DataRaw_{ele2}"][:,5] - globals()[f"DataRaw_{ele2}"][:,6] 
    PFCr = globals()[f"DataRaw_{ele2}"][:,9] - globals()[f"DataRaw_{ele2}"][:,10] 
    CA1 = globals()[f"DataRaw_{ele2}"][:,7]- globals()[f"DataRaw_{ele2}"][:,8]
    TTL = globals()[f"DataRaw_{ele2}"][:,11]
    # Stack back
    globals()[f"Good_channels_{ele2}"] = np.stack((EMG, BLAl, BLAr, PFCl, PFCr, CA1, TTL), axis = 1)#.reshape(-1, 4)
    nmberchann = globals()[f"Good_channels_{ele2}"].shape
    datalen = globals()[f"Good_channels_{ele2}"].size / nmberchann[1]
    print(datalen)
    coords = {
        'brain_areas' : np.array(['EMG', "BLAl", "BLAr", "PFCl", "PFCr", "CA1", "TTL"]),
        'duration_rec' : np.arange(0, datalen/25000, 1/25000)
    }
    # Put in xarray
    globals()[f"xrGoodChannels_{ele2}"] = xr.DataArray(coords=coords, dims=['duration_rec', 'brain_areas'])
    globals()[f"xrGoodChannels_{ele2}"].loc[:,:]  = globals()[f"Good_channels_{ele2}"]
    Good_channels = globals()[f"xrGoodChannels_{ele2}"]
    # Save datas
    #np.save(f'RawDataChannelExtracted_{ele2}.npy', Good_channels)
    list_rawchan.append(Good_channels)
    # Empty array to free memory 
    globals()[f"DataRaw_{ele2}"] = None

#  Downsample all signals to 1 kz.

In [ ]:

#list_rawchan = [xrGoodChannels_1, xrGoodChannels_2]
list_rawchan_filt = []
for ele, file in enumerate(list_rawchan):
    ele2 = ele + 1
    print(ele2)
    sampling_rate = 25000 # Hz
    new_sampling_rate = 1000 # Hz
    nmberchann = file.shape
    datalen = file.size/ nmberchann[1]
    Nmber_points = int(datalen * new_sampling_rate / sampling_rate)
    print(datalen, Nmber_points)
    globals()[f"DS_GoodChannels_{ele2}"] = signal.resample(file, Nmber_points, axis = 0)
    Good_channels_filtered = globals()[f"DS_GoodChannels_{ele2}"]
    list_rawchan_filt.append(Good_channels_filtered)
    #np.save(f'RawDataChannelExtractedDS_{ele2}.npy', Good_channels_filtered)
    globals()[f"xrGoodChannels_{ele2}"] = None

list_rawchan = None

## Filtering 

SWR: 120 - 200 Hz

In [ ]:
Sample = DS_GoodChannels_1

f_CA1 = Sample[:, 5].copy()

# Paramètres de notre filtre :
f_lowcut = 120.
f_hicut = 200.
fs = new_sampling_rate
nyq = 0.5 * fs
N = 6                 # Ordre du filtre
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
print(Wn)

# Création du filtre :
b, a = scipy.signal.butter(N, Wn, 'band')
filt_SWR_CA1 = scipy.signal.filtfilt(b, a, f_CA1)

times = np.arange(0, f_CA1.size/new_sampling_rate, 1./new_sampling_rate)

# Display. 

with Open Ephys viewer

In [ ]:
# prepare numpy array for ephyviewer
Sample2 = Sample[:, :6]
filt_SWR_CA1_2= filt_SWR_CA1[:, np.newaxis]
combined2 = np.append(Sample2, filt_SWR_CA1_2, axis=1)


In [ ]:
app = mkQApp()


sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(combined2, sample_rate, t_start, 'Signals')
win.add_view(view1)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#aa0000'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#0055ff'
view1.by_channel_params['ch5', 'color'] = '#ff5500'
view1.by_channel_params['ch6', 'color'] = '#ff5500'

view1.by_channel_params['ch0', 'gain'] = 0.00005
view1.by_channel_params['ch1', 'gain'] = 0.0002
view1.by_channel_params['ch2', 'gain'] = 0.0004
view1.by_channel_params['ch3', 'gain'] = 0.0005
view1.by_channel_params['ch4', 'gain'] = 0.0001
view1.by_channel_params['ch5', 'gain'] = 0.00005
view1.by_channel_params['ch6', 'gain'] = 0.0001

view1.by_channel_params['ch0', 'offset'] = 0.3
view1.by_channel_params['ch1', 'offset'] = 0.2
view1.by_channel_params['ch2', 'offset'] = 0.05
view1.by_channel_params['ch3', 'offset'] = -0.12
view1.by_channel_params['ch4', 'offset'] = -0.1
view1.by_channel_params['ch5', 'offset'] = -0.2
view1.by_channel_params['ch6', 'offset'] = -0.28

#Run
win.show()
app.exec_()